In [1]:
import requests
import os
from extraction.file_base_extraction import get_extractor_by_extension

QDRANT_API_BASE = "http://localhost:8002/api"
COLLECTION_NAME = "qdrant_temp"

def extract_text_chunks(file_paths):
    """파일 목록에서 텍스트를 추출하고 청크로 구성"""
    all_chunks = []
    for path in file_paths:
        if not os.path.exists(path):
            print(f"❌ 파일 없음: {path}")
            continue

        ext = os.path.splitext(path)[1].lower()
        extractor = get_extractor_by_extension(path, path)
        pages = extractor.extract_text()
        chunks = [p['text'] for p in pages if p['text'].strip()]
        all_chunks.extend(chunks)
    return all_chunks

def upload_chunks_to_qdrant(chunks, collection_name=COLLECTION_NAME):
    """Qdrant 라우터에 청크 업로드"""
    response = requests.post(
        f"{QDRANT_API_BASE}/upload_vectors",
        json={"chunks": chunks, "collection_name": collection_name}
    )
    if response.status_code != 200:
        print(f"❌ 업로드 실패: {response.status_code} {response.text}")
    else:
        print(f"✅ {len(chunks)}개 청크 업로드 완료")

def search_chunks(question, collection_name=COLLECTION_NAME):
    """질문을 기반으로 유사도 검색 수행"""
    response = requests.post(
        f"{QDRANT_API_BASE}/search_vectors",
        json={"question": question, "collection_name": collection_name}
    )
    if response.status_code != 200:
        print(f"❌ 검색 실패: {response.status_code} {response.text}")
        return []
    result = response.json().get("result", [])
    return result if isinstance(result, list) else [result]


In [2]:
# 테스트할 파일 리스트 (PDF, HWP 모두 가능)
file_paths = [
    "C:/Users/Playdata/Desktop/test/붙임1.인공지능기술고도화(신뢰성평가)지원사업재공고문.hwp",
    "C:/Users/Playdata/Desktop/test/인공지능 기술고도화 지원사업 양식.pdf"
]

# 테스트 질문
question = "사업 기간이 언제까지야?"


In [3]:
# 텍스트 추출 및 청크화
chunks = extract_text_chunks(file_paths)
print(f"📄 총 {len(chunks)}개 청크 추출 완료")

# Qdrant에 업로드
upload_chunks_to_qdrant(chunks)

# 질문 기반 유사도 검색
results = search_chunks(question)

# 결과 출력
print(f"\n🔍 질문: {question}\n")
for i, res in enumerate(results, 1):
    print(f"📄 관련 문서 {i}:\n{res}\n{'-'*50}")


📄 총 9개 청크 추출 완료
✅ 9개 청크 업로드 완료

🔍 질문: 사업 기간이 언제까지야?

📄 관련 문서 1:
'사업 기간이 언제까지야?'에 대한 관련 문서 검색 결과:

- 관련 문서 1:
(재)전북테크노파크 공고 제2025-0115호
- AI융합 지능형 농업 생태계 구축 사업 -
인공지능 기술고도화(신뢰성평가) 지원사업
모집 재공고
인공지능 기술고도화(신뢰성평가) 지원사업을 다음과 같이 재공고합니다.
2025. 4. 30.
(재)전북테크노파크원장
1
 공고 개요
1. 사업기간 : 협약일로부터 2025. 11. 30.
2. 지원내용
 가. 전북특별자치도 소재 AI 및 농업(노지 분야) 기반 기술 확보 및 AISW 품질 경쟁력 강화를 위한 평가 지원
3. 지원규모
 가. 총 지원규모 : 총 3개 지원
지원사업명
지원금액
지원수
신뢰성 평가 지원
기업당 10백만원 이내
3개
 1) 상기 지원규모 및 내용은 향후 일정에 따라 변경 될 수 있음
4. 문의처 및 접수처
 가. 문의처 : (재)전북테크노파크 AI융합팀(063-226-9070)
 나. 접수처 : https://rnd.jbtp.or.kr/h 온라인접수
 1) 지원사업의 신청, 접수, 선정 및 평가, 지원내용 등 상담 지원
 2) 유선문의 가능시간 : 평일 09:00 ~ 18:00 (점심시간 12:00 ~ 13:00 제외)
2
 세부 내용
1. 사업목적 : AI융합 지능형 농업(노지) 관련 신뢰성 확보를 통한 기업경쟁력 강화
2. 사업기간 : 협약일로부터 2025년 11월 30일까지
3. 사업내용
 가. (신뢰성평가) 지능형 농업 관련 AI기술 신뢰성 평가 비용 지원
 예) SW품질평가, 데이터품질평가, 시험평가인증(KC, 안정성 등), AI신뢰성인증, HW성능평가 등
4. 사업비 지원금 편성
 가. 사업비편성 :ICT 기금사업 사업비 산정 및 정산 등에 관한 기준 [별첨] 참조 및 사업계획서 사업비 편성표 내 예시를 참고하여 작성
 나. 사업비지급 : 협약체결 후 지원금 분할 지급
 * 내부 사정에 따